# AMP-Parkinson's Disease Progression Prediction
Use protein and peptide data measurements from Parkinson's Disease patients to predict progression of the disease.
このコンペティションの目的は、タンパク質の存在量データを用いてパーキンソン病（PD）の経過を予測することです。PDに関与するタンパク質の完全なセットは、まだ未解決の研究課題であり、予測価値を持つタンパク質は、さらに調査する価値があると思われます。このデータセットの中核は、数百人の患者から収集した脳脊髄液（CSF）サンプルの質量分析から得られたタンパク質存在量値です。各患者は、PDの重症度評価と同時に、複数年にわたり複数のサンプルを提供しました。

これを参考にコーディング予定<br>
https://www.kaggle.com/code/shimman/baseline-model-using-lgbm-regression

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tqdm
import random

from sklearn.feature_selection import SelectKBest, f_regression, RFE, RFECV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler
import lightgbm as lgbm
import xgboost as xgbm

In [24]:
path = './data/'

In [37]:
# train_data
peptide_data = pd.read_csv(f'{path}train_peptides.csv')
protein_data = pd.read_csv(f'{path}train_proteins.csv')
target_data = pd.read_csv(f'{path}train_clinical_data.csv')

data = pd.read_csv(f'{path}supplemental_clinical_data.csv')
peptide_data.shape, protein_data.shape, target_data.shape, data.shape

((981834, 6), (232741, 5), (2615, 8), (2223, 8))

# EDA

- visit_id - ID code for the visit.
- visit_month - The month of the visit, relative to the first visit by the patient.
- patient_id - An ID code for the patient.
- UniProt - The UniProt ID code for the associated protein. There are often several peptides per protein.
- Peptide - The sequence of amino acids included in the peptide. See this table for the relevant codes. Some rare - annotations may not be included in the table. The test set may include peptides not found in the train set.
- PeptideAbundance - The frequency of the amino acid in the sample.

In [39]:
peptide_data.head(7)

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7
5,55_0,0,55,O00533,TLKIENVSYQDKGNYR,23216.5
6,55_0,0,55,O00533,VIAVNEVGR,170878.0


- visit_id - ID code for the visit.
- visit_month - The month of the visit, relative to the first visit by the patient.
- patient_id - An ID code for the patient.
- UniProt - The UniProt ID code for the associated protein. There are often several peptides per protein. The test set may include proteins not found in the train set.
- NPX - Normalized protein expression. The frequency of the protein's occurrence in the sample. May not have a 1:1 - relationship with the component peptides as some proteins contain repeated copies of a given peptide.

In [40]:
protein_data.head(7)

,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.30
1,55_0,0,55,O00533,732430.00
2,55_0,0,55,O00584,39585.80
3,55_0,0,55,O14498,41526.90
4,55_0,0,55,O14773,31238.00
5,55_0,0,55,O14791,4202.71
6,55_0,0,55,O15240,177775.00


- visit_id - ID code for the visit.
- visit_month - The month of the visit, relative to the first visit by the patient.
- patient_id - An ID code for the patient.
- updrs_[1-4] - The patient's score for part N of the Unified Parkinson's Disease Rating Scale. Higher numbers indicate more severe symptoms. Each sub-section covers a distinct category of symptoms, such as mood and behavior for Part 1 and motor functions for Part 3.
- upd23b_clinical_state_on_medication - Whether or not the patient was taking medication such as Levodopa during the - UPDRS assessment. Expected to mainly affect the scores for Part 3 (motor function). These medications wear off fairly quickly (on the order of one day) so it's common for patients to take the motor function exam twice in a single month, both with and without medication.

In [41]:
target_data.head(7)

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On
5,55_18,55,18,7.0,13.0,38.0,0.0,On
6,55_24,55,24,16.0,9.0,49.0,0.0,On


In [42]:
# Creating targets
id_cols = ['visit_id','patient_id','visit_month']
target_cols = ['updrs_1','updrs_2','updrs_3','updrs_4']
month_list  =  [0,6,12,24]

# Sorting target_data by patient_id and visit_month
target_data.sort_values(['patient_id','visit_month'],inplace=True)
target_data.sort_values(['patient_id','visit_month'],inplace=True)

# Imputing upd23b_clinical_state_on_medication by 'Off'
target_data['upd23b_clinical_state_on_medication'].fillna('Off',inplace = True)
target_data['upd23b_clinical_state_on_medication'] = target_data['upd23b_clinical_state_on_medication'].str.lower().str.strip()

# Creating target variables
target_data_cln4 = pd.DataFrame()
visit_months  = target_data.visit_month.unique()
unique_patient_ids = target_data.patient_id.unique()

target_data_1 = pd.DataFrame()
for p in tqdm.tqdm(unique_patient_ids):
    p_data = target_data[target_data['patient_id'] == p].drop(columns = ['visit_id','upd23b_clinical_state_on_medication']).copy()
    visit_months = list(p_data.visit_month.unique())
    for v in visit_months:
        for i in [0,6,12,24]:
            temp = p_data[p_data['visit_month']==v+i].copy()
            temp['visit_month_1'] = i
            temp['visit_month'] = v
            target_data_1 = pd.concat([target_data_1,temp],axis = 0)

# Imputing updrs_4 with 0
target_data_1.updrs_4.fillna(0,inplace = True)
target_data_1.reset_index(drop = True,inplace = True)

# Dropping missing values from the dataset
target_data_1 = target_data_1.dropna().drop_duplicates()

# Creating visit_id
target_data_1['visit_id'] = target_data_1['patient_id'].astype(str) + '_' + target_data_1['visit_month'].astype(str)
target_data_1 = target_data_1[id_cols + target_cols + ['visit_month_1']].copy()
target_data_1.shape

100%|██████████| 248/248 [00:06<00:00, 38.29it/s]


(7567, 8)

In [43]:
target_data_1.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,visit_month_1
0,55_0,55,0,10.0,6.0,15.0,0.0,0
1,55_0,55,0,8.0,10.0,34.0,0.0,6
2,55_0,55,0,10.0,10.0,41.0,0.0,12
3,55_0,55,0,16.0,9.0,49.0,0.0,24
4,55_3,55,3,10.0,7.0,25.0,0.0,0


In [44]:
def create_features(peptides_data,protein_data,target_data_1):

    # Processing peptides data and protein data 
    pep_pro_data = peptides_data.merge(protein_data,on = id_cols + ['UniProt'],how = 'left')
#     peptides_list = ['QGVNDNEEGFFSAR','AGLAASLAGPHSIVGR','MELERPGGNEITR','KTSLEDFYLDEER','RYIETDPANRDR','EWVAIESDSVQPVPR','LQDLYSIVR','AIQLTYNPDESSKPNMIDAATLK']
    peptides_list = peptides_data.Peptide.unique().tolist()

    pep_pro_data =  pep_pro_data[pep_pro_data.Peptide.isin(peptides_list)].copy()

    # Some features based on id cols 
    visit_month_summary = pep_pro_data.groupby('visit_month')[['PeptideAbundance','NPX']].agg(['min','max','mean','median','sum','std'])
    visit_month_summary.columns = [i+'_'+j for i,j in visit_month_summary.columns.tolist()]

    # Calculating ratio columns of NPX and PeptideAbundance
    for i in ['min','max','mean','median','sum','std']:
        visit_month_summary['pepab_to_npx_ratio_'+i] = visit_month_summary[f'PeptideAbundance_{i}']/visit_month_summary[f'NPX_{i}']

    # Patient level summary
    patient_summary = pep_pro_data.groupby('patient_id')[['PeptideAbundance','NPX']].agg(['min','max','mean','median','sum','std'])
    patient_summary.columns = [i+'_'+j for i,j in patient_summary.columns.tolist()]

    # Calculating ratio columns of NPX and PeptideAbundance
    for i in ['min','max','mean','median','sum','std']:
        patient_summary['pepab_to_npx_ratio_'+i] = patient_summary[f'PeptideAbundance_{i}']/patient_summary[f'NPX_{i}']

    # Creating PeptideAbundance / NPX
    pep_pro_data['pepab_to_npx_ratio'] = pep_pro_data['PeptideAbundance']/pep_pro_data['NPX']

    temp = pep_pro_data.pivot(index= id_cols,columns = 'Peptide',values = ['PeptideAbundance','NPX','pepab_to_npx_ratio'])
    temp.columns = [j+'_'+i for i,j in temp.columns]
    temp.reset_index(inplace = True)
    temp.fillna(0,inplace = True)

    pep_pro_features = temp.merge(patient_summary.reset_index(),on = 'patient_id').merge(
        visit_month_summary.reset_index(),on = 'visit_month',suffixes = ('_patient','_visit_month'))
    
    if target_data_1 is None:
        return pep_pro_features
    else:
        pep_pro_features = pep_pro_features.merge(target_data_1,on = id_cols,how = 'inner')   
        return pep_pro_features

In [45]:
def smape(y_true, y_pred):
    y_true = 1+y_true
    y_pred = 1+y_pred
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

smape_ = make_scorer(smape,greater_is_better=False)

def feature_selector(X,y,k,n_splits,groups):

    # scaler = StandardScaler()
    # scaler.fit(X)
    # X = pd.DataFrame(scaler.transform(X),columns = X.columns.tolist(), index = X.index.tolist())
    
    # First feature selection using correlation and second using RFECV
    selector = SelectKBest(score_func=f_regression, k=k)
    feature_names = X.columns.tolist()

    # fit selector to the data and transform the feature matrix
    X_selected = selector.fit_transform(X, y)

    # get the indices of the selected features
    selected_indices = selector.get_support(indices=True)
    selected_features = [feature_names[i] for i in selected_indices]
    
#     print(selected_features)

    # instantiate a ML model
    lgbm = lgb.LGBMRegressor(n_jobs=-1)
    # xgbm = xgb.XGBRegressor()

    # Initialize GroupKFold with 10 folds
    kf = GroupKFold(n_splits=n_splits)

    rfecv = RFECV(estimator=lgbm, min_features_to_select=10,cv = kf.split(X, y, groups), step = 1,scoring = smape_)

    # fit RFE to the data
    rfecv.fit(X[selected_features], y)

    feature_rank = pd.DataFrame({'features' : selected_features,'rank':rfecv.ranking_}).sort_values('rank')
    selected_features2 = feature_rank[feature_rank['rank'] < 2].features.tolist()
    
    return selected_features

In [46]:
# Loading libraries
from sklearn.model_selection import GroupKFold
from lightgbm import LGBMRegressor

In [50]:
# print(sel_features )
sel_features = ['AIQLTYNPDESSKPNMIDAATLK_PeptideAbundance', 'EWVAIESDSVQPVPR_PeptideAbundance', 'KTSLEDFYLDEER_PeptideAbundance', 'LQDLYSIVR_PeptideAbundance', 'MELERPGGNEITR_PeptideAbundance', 'QGVNDNEEGFFSAR_PeptideAbundance', 'AGLAASLAGPHSIVGR_NPX', 'AIQLTYNPDESSKPNMIDAATLK_NPX', 'EWVAIESDSVQPVPR_NPX', 'KTSLEDFYLDEER_NPX', 'LQDLYSIVR_NPX', 'MELERPGGNEITR_NPX', 'QGVNDNEEGFFSAR_NPX', 'EWVAIESDSVQPVPR_pepab_to_npx_ratio', 'LQDLYSIVR_pepab_to_npx_ratio', 'RYIETDPANRDR_pepab_to_npx_ratio', 'PeptideAbundance_min_patient', 'PeptideAbundance_mean_patient', 'PeptideAbundance_median_patient', 'PeptideAbundance_std_patient', 'NPX_min_patient', 'NPX_max_patient', 'NPX_mean_patient', 'NPX_sum_patient', 'NPX_std_patient', 'pepab_to_npx_ratio_max_patient', 'pepab_to_npx_ratio_median_patient', 'pepab_to_npx_ratio_std_patient', 'PeptideAbundance_min_visit_month', 'PeptideAbundance_sum_visit_month', 'NPX_min_visit_month', 'NPX_max_visit_month', 'NPX_median_visit_month', 'NPX_sum_visit_month', 'NPX_std_visit_month', 'pepab_to_npx_ratio_min_visit_month', 'pepab_to_npx_ratio_max_visit_month', 'pepab_to_npx_ratio_median_visit_month', 'pepab_to_npx_ratio_std_visit_month', 'visit_month_1']


In [51]:
# Function to create lgb models

def train_models_lgb(model_data,target_cols,sel_features, params):    

    groups = model_data['patient_id']

    # Initialize GroupKFold with 10 folds
    kf = GroupKFold(n_splits=10)
    
    # Initialize a dictionary to store the models and scores
    models = {}
    scores_trn = {} ; scores_val = {}
    scores2_trn = {} ; scores2_val = {}
    
    # Loop through each combination of hyperparameters
    for i, target in enumerate(target_cols):

        X = model_data[sel_features].copy()
        y = model_data[target].copy() 

        # Initialize variables for storing predictions and actual values
        predictions_val = np.zeros(len(X))
        predictions2_val = np.zeros(len(X))
        actuals_val = np.zeros(len(X))
        
        predictions_trn = np.zeros(len(X))
        predictions2_trn = np.zeros(len(X))
        actuals_trn = np.zeros(len(X))
        
        model_temp = []
        # Loop through each fold
        for train_idx, test_idx in kf.split(X, y, groups):
            # Get training and validation data for the fold
            X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]
            
            # Initialize LightGBM Regressor
            model = LGBMRegressor(**params, random_state=2023)

            # Train the model
            model.fit(X_train, y_train)

            # Make predictions on validation data for the fold
            fold_preds = model.predict(X_val)
            trn_preds = model.predict(X_train)

            # Store predictions and actuals for the fold
            predictions_val[test_idx] = fold_preds
            actuals_val[test_idx] = y_val

            # Store predictions and actuals for the fold
            predictions_trn[train_idx] = trn_preds
            actuals_trn[train_idx] = y_train

            if target == 'updrs_4':
                predictions2_val[test_idx] = 0
                predictions2_trn[train_idx] = 0
            else:
                predictions2_val[test_idx] = fold_preds
                predictions2_trn[train_idx] = trn_preds
                
            model_temp += [model]

                
            
        # Calculate the validation score for the model
        score_val = smape(actuals_val,predictions_val)   
        score2_val  = smape(actuals_val,predictions2_val)
        
        score_trn  = smape(actuals_trn,predictions_trn)
        score2_trn  = smape(actuals_trn,predictions2_trn)

        # Print the train score for the model
        print(f"\nModel for {target} train score: {score_trn:.3f}")
        
        # Print the validation score for the model
        print(f"Model for {target} validation score: {score_val:.3f}")
        
        if target == 'updrs_4':
            
            print(f"\nModel for {target} train score when pred is 0: {score2_trn:.3f}")
            # Print the validation score for the model
            print(f"Model for {target} validation score when pred is 0: {score2_val:.3f}")
        
        # Store the model and score in the dictionary
        models[target] = model_temp
        scores_trn[target] = score_trn
        scores2_trn[target] = score2_trn
        scores_val[target] = score_val
        scores2_val[target] = score2_val
    
    print(f"\nTrain score for all models : {np.mean(list(scores_trn.values())):3f}")
    print(f'Train score for all models when updrs_4 pred is 0 : {np.mean(list(scores2_trn.values())):3f}')
    
    print(f"\nValidation score for all models : {np.mean(list(scores_val.values())):3f}")
    
    print(f'Validation score for all models when updrs_4 pred is 0 : {np.mean(list(scores2_val.values())):3f}')

    return models 

In [52]:
# Variables for model
params = {
    'learning_rate' : 0.0005,
    'n_estimators' : 1500,
    'reg_alpha' : 5,
    'reg_lambda' : 5,
    'min_child_samples' : 30,
    'colsample_bytree' : 0.6,
    'subsample':0.6,
    'num_leaves' : 10,
    'max_depth' : 2
    }

In [53]:
# Creating features & Building model
model_data = create_features(peptide_data,protein_data,target_data_1)
lgb_models = train_models_lgb(model_data,target_cols,sel_features,params)


Model for updrs_1 train score: 54.826
Model for updrs_1 validation score: 57.076

Model for updrs_2 train score: 71.201
Model for updrs_2 validation score: 73.679

Model for updrs_3 train score: 73.133
Model for updrs_3 validation score: 75.497

Model for updrs_4 train score: 71.335
Model for updrs_4 validation score: 73.244

Model for updrs_4 train score when pred is 0: 31.771
Model for updrs_4 validation score when pred is 0: 31.771

Train score for all models : 67.623660
Train score for all models when updrs_4 pred is 0 : 57.732819

Validation score for all models : 69.874173
Validation score for all models when updrs_4 pred is 0 : 59.505996


## Testing the code

In [57]:
# test data
test_proteins = pd.read_csv(f'{path}example_test_files/test_proteins.csv')
test_peptides = pd.read_csv(f'{path}example_test_files/test_peptides.csv')
test = pd.read_csv(f'{path}example_test_files/test.csv')
sample_submission = pd.read_csv(f'{path}example_test_files/sample_submission.csv')

In [58]:
sample_submission

,prediction_id,rating,group_key
0,3342_0_updrs_1_plus_0_months,0,0
1,3342_0_updrs_1_plus_6_months,0,0
2,3342_0_updrs_1_plus_12_months,0,0
3,3342_0_updrs_1_plus_24_months,0,0
4,3342_0_updrs_2_plus_0_months,0,0
...,...,...,...
59,50423_6_updrs_3_plus_24_months,0,6
60,50423_6_updrs_4_plus_0_months,0,6
61,50423_6_updrs_4_plus_6_months,0,6
62,50423_6_updrs_4_plus_12_months,0,6


In [59]:
def get_predictions(test,peptides_data,protein_data,sample_submission,models,sel_features):

    submission_ids  = test.visit_id.tolist()
    features = create_features(peptides_data,protein_data,None)
#     return features

    test1 = test.copy()
    test1 = test1[['patient_id','visit_month']].drop_duplicates().copy()
    patients = test1.patient_id.unique()
    visit_months=test1.visit_month.unique()
    test1['visit_month_1'] = test1['visit_month']
    test_data = pd.DataFrame()
    for patient in patients:
        for month in [0,6,12,24]:
            p_data = test1[test1['patient_id'] == patient].copy()
            p_data['visit_month_1'] = month
            test_data = pd.concat([test_data,p_data],axis=0)
    test_data.sort_values(by = ['patient_id','visit_month','visit_month_1'],inplace=  True)
    test_data = test_data.reset_index(drop = True)
#     return test_data
    
    features = pd.merge(test_data,features,on = ['patient_id','visit_month'],how = 'inner')
    
    # Converting infinite values to pd.NA
    features = features.replace([np.inf, -np.inf], pd.NA)
    
    # Imputing missing values by 0
    features.fillna(0,inplace = True)
    
#     return test_data

    missing_features = list(set(sel_features).difference(features.columns.tolist()))
    features[missing_features] = 0
    print('missing features : \n',missing_features )

    # Making predictions from all the models
    pred_submission = pd.DataFrame(index = range(features.shape[0]))
    
    for key,value in models.items():
        pred_feats = features[sel_features]        
        pred_train_temp = pd.DataFrame(np.mean(np.array([np.array(mod.predict(pred_feats)) for mod in value]),axis = 0))
        pred_train_temp.columns = [key]
        pred_submission = pred_submission.join(pred_train_temp)
    
    pred_submission = pred_submission[models.keys()].copy()
    pred_submission.index = features['patient_id'].astype(str) + '_' +features['visit_month'].astype(str) + '_'+features['visit_month_1'].apply(lambda x: 'plus_'+str(x) +'_months')

    pred_submission['updrs_1'] = np.clip(pred_submission['updrs_1'].abs(),0,52)
    pred_submission['updrs_2'] = np.clip(pred_submission['updrs_2'].abs(),0,52)
    pred_submission['updrs_3'] = np.clip(pred_submission['updrs_3'].abs(),0,132)
    pred_submission['updrs_4'] = 0

    pred_submission = pred_submission.stack().reset_index().rename(columns = {'level_0':'prediction_id','level_1':'target',0:'rating'})
    pred_submission['prediction_id'] = pred_submission[['prediction_id','target']].apply(lambda x: '_'.join(x[0].split('_')[:2] + [x[1]] + x[0].split('_')[2:]) ,axis= 1)
    pred_submission.drop(columns = ['target'],inplace= True)
#     pred_submission['group_key'] = pred_submission['prediction_id'].apply(lambda x: x.split('_')[1]).astype('str')
    pred_submission.reset_index(drop = True,inplace = True)
    sample_submission = sample_submission.drop(columns = ['rating']).copy()
    sample_submission = sample_submission.merge(pred_submission,on = ['prediction_id'],how = 'left')
    
    
    # Calculating medians
    temp  = pred_submission.copy()
    temp['target'] = temp.prediction_id.apply(lambda x: '_'.join(x.split('_')[2:]))
    pred_medians = temp.groupby('target')['rating'].median()

    sample_submission['rating'] = sample_submission['rating'].fillna(
        
        sample_submission.prediction_id.apply(lambda x: '_'.join(x.split('_')[2:])).map(pred_medians)
    
    )
    
    sample_submission['rating'] = sample_submission['rating'].apply(lambda x: np.ceil(x))

    return sample_submission

In [60]:
temp  = get_predictions(test,test_peptides,test_proteins,sample_submission,lgb_models,sel_features)

missing features : 
 ['RYIETDPANRDR_pepab_to_npx_ratio']


In [66]:
import sys
sys.path.append('/data/amp_pd_peptide/')

import amp_pd_peptide
amp_pd_peptide.make_env.func_dict['__called__'] = False
env = amp_pd_peptide.make_env()

iter_test = env.iter_test() 

ModuleNotFoundError: No module named 'amp_pd_peptide'